## Scalable Serverless Architectures (9 Points)

i. (2 Points) Use Boto3 to store the raw JSON data from each survey submission as a JSON file in S3. You can label your files however you like, but you’ll want to make sure that you use a unique identifier (e.g. time stamp, followed by user ID).

 ii. (1 Point) Use AWS Comprehend to determine the dominant sentiment of the participant’s submitted text response. This should be a string, such as “POSITIVE” or “NEGATIVE.”

 iii. (2 Points) Insert/Update the participant’s record into a DynamoDB database, with the User’s ID number as the partition key, along with their current sentiment (determined above), their current self-reported mood, as well as the number of times the user has completed a survey overall (including the current survey you are processing).

 iv. (1 Point) Finally, your Lambda function should return the partici- pant’s current sentiment back to the app (i.e. the location where the function was invoked) for further use within the app.

In [1]:
!pip install boto3

In [1]:
import boto3
import time
import pandas as pd
import json
from boto3.dynamodb.conditions import Key
from concurrent.futures import ThreadPoolExecutor

In [2]:
s3 = boto3.resource('s3')
s3_resource = boto3.resource('s3')

### Load bucket

In [3]:
test_json = {
            "user_id": "0001",
            "timestamp": "092821120000",
            "mood": 2,
            "text": "I am having a really bad day..."}
json_file = json.dumps(test_json, ensure_ascii=False)

In [4]:
json_file

'{"user_id": "0001", "timestamp": "092821120000", "mood": 2, "text": "I am having a really bad day..."}'

In [5]:
test_json = {
            "user_id": "0001",
            "timestamp": "092821120000",
            "mood": 2,
            "text": "I am having a really bad day..."}
json_file = json.dumps(test_json, ensure_ascii=False)

# Create Bucket:
s3.create_bucket(Bucket='macs30123-andresc')

# Put object into bucket:
object = s3.Object('macs30123-andresc', '{}_{}.txt'.format(test_json['timestamp'],test_json['user_id']))
object.put(Body=json_file)

{'ResponseMetadata': {'RequestId': 'NPTBM4QMCT1PCZDJ',
  'HostId': 'cSxibiMsFlknjCzZlPcvdRy9Hi9TELc04Hxdff9IWHA7Kyo2rIwZdB+WY0zFdVNbik6dthUzvWA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'cSxibiMsFlknjCzZlPcvdRy9Hi9TELc04Hxdff9IWHA7Kyo2rIwZdB+WY0zFdVNbik6dthUzvWA=',
   'x-amz-request-id': 'NPTBM4QMCT1PCZDJ',
   'date': 'Tue, 02 Nov 2021 15:44:49 GMT',
   'etag': '"6dc22ff423a5668fe54a457ccc747bd7"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"6dc22ff423a5668fe54a457ccc747bd7"'}

In [7]:
comprehend = boto3.client('comprehend')

### Get sentiment

In [8]:
body = object.get()['Body'].read()
contents = json.loads(body.decode('utf-8'))
sentiment = comprehend.detect_sentiment(Text = contents['text'], LanguageCode = 'en') 

In [9]:
contents

{'user_id': '0001',
 'timestamp': '092821120000',
 'mood': 2,
 'text': 'I am having a really bad day...'}

In [29]:
sentiment['Sentiment']

{'Sentiment': 'NEGATIVE',
 'SentimentScore': {'Positive': 0.009881318546831608,
  'Negative': 0.8181014060974121,
  'Neutral': 0.13082244992256165,
  'Mixed': 0.04119482636451721},
 'ResponseMetadata': {'RequestId': '0559e322-536a-4903-9216-d5d12e31378d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0559e322-536a-4903-9216-d5d12e31378d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '163',
   'date': 'Mon, 01 Nov 2021 18:06:07 GMT'},
  'RetryAttempts': 0}}

### Upload into DynamoDB dataset

In [27]:
dynamodb = boto3.resource('dynamodb')

table = dynamodb.create_table(
    TableName='tweet_sentiment',
    KeySchema=[
        {
            'AttributeName': 'username',
            'KeyType': 'HASH'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'username',
            'AttributeType': 'S'
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    }    
)

# Wait until AWS confirms that table exists before moving on
table.meta.client.get_waiter('table_exists').wait(TableName='tweet_sentiment')

# get data about table (should currently be no items in table)
print(table.item_count)
print(table.creation_date_time)

0
2021-11-01 13:18:12.958000-05:00


In [31]:
contents

{'user_id': '0001',
 'timestamp': '092821120000',
 'mood': 2,
 'text': 'I am having a really bad day...'}

In [32]:
table.put_item(
   Item={
        'username': contents['user_id'],
        'sentiment': sentiment['Sentiment'],
        'mood': contents['mood'],
        'survey_count': 0
    }
)

{'ResponseMetadata': {'RequestId': 'RN98G5ERTC7C16I4GMQVNICKI3VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 01 Nov 2021 18:21:10 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'RN98G5ERTC7C16I4GMQVNICKI3VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [50]:
try: 
    response = table.get_item(
    Key={
        'username': '0001'
    }
    )
except dynamodb.exceptions.DynamoDBKeyNotFoundError:
    print("No item here")

In [57]:
response['Item']['survey_count']+2

Decimal('2')

In [62]:
table.update_item(
    Key={
        'username': '0001'
    },
    UpdateExpression='SET num_tweets = :val1',
    ExpressionAttributeValues={
        ':val1': 2
    }
)

{'ResponseMetadata': {'RequestId': 'CNL1PMQJAF2VMCA3QO3RR1K30FVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 01 Nov 2021 19:00:56 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'CNL1PMQJAF2VMCA3QO3RR1K30FVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [63]:
response = table.get_item(
Key={
'username': '0001'
}
)

In [66]:
response['Item']

{'mood': Decimal('2'),
 'survey_count': Decimal('0'),
 'username': '0001',
 'num_tweets': Decimal('2'),
 'sentiment': 'NEGATIVE'}

### Testing with Lambda

In [12]:
aws_lambda = boto3.client('lambda')

test_json = {
            "user_id": "0001",
            "timestamp": "092821120000",
            "mood": 2,
            "text": "I am having a really bad day..."}

# run synchronously:
r = aws_lambda.invoke(FunctionName='tweet_sentiment_analysis',
                      InvocationType='RequestResponse',
                      Payload=json.dumps(test_json))

json.loads(r['Payload'].read()) # print out response

{'statusCode': 200, 'body': 'NEGATIVE'}